# How to complete this assignment
First, download [this Kaggle dataset](https://www.kaggle.com/hugomathien/soccer) and extract *sqlite* database. You may need to register at https://www.kaggle.com/ beforehand. Then complete 15 graded tasks below, the score is given in brackets. Finally submit the resulting `.ipynb` file to rs-app Auto-test.

- Do not delete or rename the variables given before the inscription `#your code here`, they are needed for the correct verification.
- Do not change the code in the last Notebook cell, it is required for the server check.
- Your Notebook must run completely without errors to be graded! Please check everything before submission by going *Cell -> Run All*

## Some important notes
- If you need to **calculate the number of "something"** that means we expect you to assign an Integer to the given variable
- If you need to **make a list of "something"** we expect you to assign a Python list with appropriate values to the given variable
- If you need to find a **specifiс player, day of the week, team, etc.** we expect you to assign a String with the full name of the entity to the given variable (`player_name`, day of week full name, `team_name`, etc.)

In [48]:
import sqlite3
import pandas as pd
import os

pd.set_option('display.max_column', None)

In [49]:
# Leave that code unchanged, it is required for the server check!
db = sqlite3.connect(os.environ.get("DB_PATH") or 'database.sqlite')

In [50]:
# You may load the data from SQL table directly to the Pandas dataframe as
player_data = pd.read_sql("SELECT * FROM Player;", db)

In [51]:
player_data.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


**Task 1 (0.25 point).** Calculate the number of players with a height between 180 and 190 inclusive

In [52]:
players_180_190 = player_data['height'].between(180,190).sum()

**Task 2 (0.25 point).** Calculate the number of players born in 1980. <br>
**Hint:** you may want to cast your 'birthday' column to DateTime type by [pandas.to_datetime](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)

In [53]:
players_1980 = (pd.to_datetime(player_data['birthday']).dt.year == 1980).sum()

**Task 3 (0.25 point).** Make a list of the top 10 players with the highest weight sorted in descending order. If there are several players with the same weight put them in the lexicographic order by name.

In [54]:
highest_players = player_data.sort_values(['weight', 'player_name'], ascending=(False, True))['player_name'].head(10).to_list()

**Task 4 (0.5 point).** Make a list of tuples containing years along with the number of players born in that year from 1980 up to 1990. <br>
**Structure example**: [(1980, 123), (1981, 140) ..., (1990, 83)] -> There were born 123 players in 1980, there were born 140 players in 1981 and etc.

In [55]:
player_data1 = player_data
player_data1['year'] = pd.to_datetime(player_data['birthday']).dt.year
player_year_series = player_data1[player_data1['year'].between(1980,1990)].groupby('year').size()
years_born_players = list(zip(player_year_series.index, player_year_series))

**Task 5 (0.5 point).** Calculate the mean and the standard deviation of the players' **height** with the name **Adriano**. <br>
**Note:** Name is represented by the first part of `player_name`.

In [56]:
adriano_mean, adriano_std = player_data[player_data['player_name'].str.startswith('Adriano')]['height'].agg(['mean', 'std']).values

**Task 6 (0.75 point).** How many players were born on each day of the week? Find the day of the week with the minimum number of players born.

In [57]:
player_data2 = player_data
player_data2['day_of_birth'] = pd.to_datetime(player_data['birthday']).dt.day_name()
dow_with_min_players_born = player_data2.groupby('day_of_birth').size().idxmin()

**Task 7 (0.75 point).** Find a league with the most matches in total. If there are several leagues with the same amount of matches, take the first in the lexical order.

In [58]:
match_data = pd.read_sql("SELECT * FROM Match;", db)
league_data = pd.read_sql("SELECT * FROM League;", db)
dfg = match_data.groupby('league_id', as_index=False, sort=False).size()
max_matches = dfg['size'].max()
mdfg = dfg[dfg['size'] == max_matches]
mdfg1 = mdfg.rename(columns={'league_id':'id'})

league_most_matches = pd.merge(league_data, mdfg1).sort_values('name').iloc[0]['name']

**Task 8 (1.25 point).** Find a player who participated in the largest number of matches during the whole match history. Assign a `player_name` to the given variable

In [59]:
home_player = 'home_player_'
away_player = 'away_player_'
home_players = [item+str(index) for index, item in enumerate(list([home_player]*11), start=1)]
away_players = [item+str(index) for index,
                item in enumerate(list([away_player]*11), start=1)]
players = home_players + away_players
# match_data.dtypes
ps = [match_data[item].dropna().astype('int64') for item in players]
most_freq = pd.concat(ps).mode()
max_matches_player = player_data[player_data['player_api_id'] ==
int(most_freq)].iloc[0]['player_name']


**Task 9 (1.5 point).** List top-5 tuples of most correlated **player's characteristics** in the descending order of the absolute [Pearson's coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) value.

**Note 1:** Players characteristics are all the columns in `Player_Attributes` table except `[id, player_fifa_api_id, player_api_id, date, preferred_foot, attacking_work_rate, defensive_work_rate]`). <br>
**Note 2:** Exclude duplicated pairs from the list. E.g. ('gk_handling', 'gk_reflexes') and ('gk_reflexes', 'gk_handling') are duplicates, leave just one of them in the resulting list.

**Hint:** You may use [dataframe.corr()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) for calculating pairwise Pearson correlation.

In [69]:
player_attributes_data = pd.read_sql("SELECT * FROM Player_Attributes;", db)
pure_player_attributes_data = player_attributes_data.drop(columns=['id', 'player_fifa_api_id', 'player_api_id', 'date', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate'])
pure_player_attributes_data.corr()
# top_correlated_features = [(0,0),(0,0),(0,0)]

,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
overall_rating,1.000000,0.766757,0.357699,0.329298,0.314099,0.458361,0.361470,0.354324,0.357772,0.349592,0.435018,0.444257,0.245655,0.254841,0.239896,0.769246,0.160364,0.427996,0.259023,0.327456,0.318661,0.392382,0.323934,0.250370,0.370019,0.431137,0.393189,0.133377,0.165349,0.128889,0.027976,0.004410,0.025682,0.005709,0.005687
potential,0.766757,1.000000,0.277755,0.286684,0.207706,0.382605,0.301302,0.338675,0.296073,0.263358,0.343716,0.401368,0.338044,0.340054,0.292950,0.580235,0.201974,0.325697,0.174819,0.260509,0.126931,0.312808,0.164815,0.165730,0.327829,0.378901,0.316414,0.055758,0.083822,0.063976,-0.011250,0.004742,0.089205,0.003406,0.003962
crossing,0.357699,0.277755,1.000000,0.576860,0.369747,0.790150,0.637891,0.809061,0.789132,0.707576,0.681959,0.807302,0.599012,0.579409,0.599248,0.384514,0.518518,0.656280,0.021320,0.564360,-0.070291,0.715925,0.323599,0.305952,0.682113,0.693227,0.571473,0.235159,0.285265,0.275467,-0.603898,-0.595610,-0.349559,-0.597625,-0.601580
finishing,0.329298,0.286684,0.576860,1.000000,0.373536,0.579365,0.851339,0.784659,0.691201,0.632248,0.337601,0.719709,0.528942,0.509506,0.554235,0.354473,0.394012,0.726982,0.008953,0.346688,-0.053048,0.806005,0.043714,-0.149366,0.798441,0.651455,0.720388,-0.284603,-0.229699,-0.260890,-0.479162,-0.465099,-0.287681,-0.470673,-0.473173
heading_accuracy,0.314099,0.207706,0.369747,0.373536,1.000000,0.548700,0.391638,0.400494,0.321120,0.307587,0.358634,0.551049,0.200077,0.267273,0.069220,0.295348,0.078953,0.540868,0.286803,0.477466,0.493051,0.406758,0.575705,0.452849,0.411554,0.337184,0.431336,0.461543,0.480624,0.441769,-0.665365,-0.647876,-0.391540,-0.647738,-0.651373
short_passing,0.458361,0.382605,0.790150,0.579365,0.548700,1.000000,0.640253,0.787994,0.732295,0.692461,0.797488,0.890042,0.503266,0.491380,0.510591,0.459289,0.461714,0.721767,0.060146,0.609956,0.092084,0.728791,0.453783,0.424288,0.677577,0.765336,0.609537,0.350171,0.415678,0.380990,-0.693313,-0.689528,-0.413317,-0.690702,-0.692926
volleys,0.361470,0.301302,0.637891,0.851339,0.391638,0.640253,1.000000,0.784366,0.752248,0.682751,0.414123,0.749557,0.513250,0.494090,0.559756,0.397233,0.415812,0.746484,0.023230,0.382649,-0.036108,0.814433,0.127922,-0.037604,0.778337,0.689852,0.711830,-0.168972,-0.106926,-0.126398,-0.508823,-0.487015,-0.279030,-0.490968,-0.492979
dribbling,0.354324,0.338675,0.809061,0.784659,0.400494,0.787994,0.784366,1.000000,0.810404,0.705863,0.574307,0.900768,0.698161,0.669503,0.703083,0.377661,0.546064,0.744135,0.008559,0.525359,-0.111598,0.806329,0.202770,0.106738,0.793309,0.732742,0.657698,0.004128,0.066958,0.046042,-0.652739,-0.650525,-0.427115,-0.653410,-0.656030
curve,0.357772,0.296073,0.789132,0.691201,0.321120,0.732295,0.752248,0.810404,1.000000,0.797411,0.585693,0.798737,0.549363,0.516899,0.619253,0.392910,0.493112,0.694749,-0.016958,0.454334,-0.114361,0.783548,0.203647,0.136265,0.720219,0.726943,0.648502,0.033682,0.095210,0.081215,-0.557165,-0.545760,-0.333487,-0.550659,-0.552295
free_kick_accuracy,0.349592,0.263358,0.707576,0.632248,0.307587,0.692461,0.682751,0.705863,0.797411,1.000000,0.599103,0.719743,0.430791,0.394848,0.504998,0.368100,0.430410,0.682790,-0.033263,0.416123,-0.056184,0.772652,0.232429,0.178005,0.654151,0.697390,0.665831,0.074506,0.134508,0.106969,-0.498800,-0.492137,-0.271076,-0.494807,-0.496332


**Task 10 (2 points).** Find top-5 most similar players to **Neymar** whose names are given. The similarity is measured as [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between vectors of players' characteristics (described in the task above). Put their names in a vector in ascending order by Euclidean distance and sorted by `player_name` if the distance is the same<br>
**Note 1:** There are many records for some players in the `Player_Attributes` table. You need to take the freshest data (characteristics with the most recent `date`). <br>
**Note 2:** Use pure values of the characteristics even if you are aware of such preprocessing technics as normalization. <br>
**Note 3:** Please avoid using any built-in methods for calculating the Euclidean distance between vectors, think about implementing your own.

In [61]:
neymar_similarities = 0

**Task 11 (1 point).** Calculate the number of home matches played by the **Borussia Dortmund** team in **Germany 1. Bundesliga** in season **2008/2009**

In [62]:
borussia_bundesliga_2008_2009_matches = 0

**Task 12 (1 point).** Find a team having the most matches (both home and away!) in the **Germany 1. Bundesliga** in **2008/2009** season. Return number of matches.

In [63]:
team_most_matches_bundesliga_2008_2009 = 0

**Task 13 (1 point).** Count total number of **Arsenal** matches (both home and away!) in the **2015/2016** season which they have won. <br><br>
**Note:** Winning a game means scoring **more** goals than an opponent.

In [64]:
arsenal_won_matches_2015_2016 = 0

**Task 14 (2 points).** Find a team with the highest win rate in the **2015/2016** season. Win rate means won matches / all matches. If there are several teams with the highest win rate return the first by name in lexical order

In [65]:
team_highest_winrate_2015_2016 = 0

**Task 15 (2 points).** Determine the team with the maximum days' gap between matches in **England Premier League 2010/2011 season**. Return number of days in that gap. <br>
**Note**: a *gap* means the number of days between two consecutive matches of the same team.

In [66]:
highest_gap_england_2010_2011 = 0

### Warning! Do not change anything in the area below

In [67]:
with open('student_answers.txt', 'w') as file:
    file.write(f"{players_180_190}\n")
    file.write(f"{players_1980}\n")
    file.write(f"{highest_players}\n")
    file.write(f"{years_born_players}\n")
    file.write(f"{round(adriano_mean, 3)} {round(adriano_std, 3)}\n")
    file.write(f"{dow_with_min_players_born}\n")
    file.write(f"{league_most_matches}\n")
    file.write(f"{max_matches_player}\n")
    file.write(f"{';'.join(['%s,%s' % tup for tup in top_correlated_features])};\n")
    file.write(f"{neymar_similarities}\n")
    file.write(f"{borussia_bundesliga_2008_2009_matches}\n")
    file.write(f"{team_most_matches_bundesliga_2008_2009}\n")
    file.write(f"{arsenal_won_matches_2015_2016}\n")
    file.write(f"{team_highest_winrate_2015_2016}\n")
    file.write(f"{highest_gap_england_2010_2011}\n")